<a href="https://colab.research.google.com/github/manmeet3591/python_class/blob/master/xarray_tutorial/cape_cin_wrf_python_lcl_lfc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install condacolab
import condacolab
condacolab.install()
condacolab.install_anaconda()
import numpy as np
import xarray as xr
!conda install --yes -c conda-forge cartopy
!pip install metpy
# Any import of metpy will activate the accessors
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.units import units

  Using cached condacolab-0.1.3-py3-none-any.whl (6.8 kB)
✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cartopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    cartopy-0.19.0.post1       |   py37h0c48da3_0         1.8 MB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_0         3.1 MB  conda-forge
    cycler-0.10.0              |             py_2           9 KB  conda-forge
    freetype-2

In [ ]:
# Open the netCDF file as a xarray Dataset
data = xr.open_dataset(get_test_data('irma_gfs_example.nc', False))

# View a summary of the Dataset
data

<xarray.Dataset>
Dimensions:                              (isobaric1: 21, isobaric3: 31, latitude: 81, longitude: 131, time1: 9)
Coordinates:
  * time1                                (time1) datetime64[ns] 2017-09-05T12...
    reftime                              datetime64[ns] ...
  * latitude                             (latitude) float32 50.0 49.5 ... 10.0
  * isobaric3                            (isobaric3) float64 100.0 ... 1e+05
  * isobaric1                            (isobaric1) float64 1e+04 ... 1e+05
  * longitude                            (longitude) float32 250.0 ... 315.0
Data variables:
    Vertical_velocity_pressure_isobaric  (time1, isobaric1, latitude, longitude) float32 ...
    Relative_humidity_isobaric           (time1, isobaric3, latitude, longitude) float32 ...
    Temperature_isobaric                 (time1, isobaric3, latitude, longitude) float32 ...
    u-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) float32 ...
    v-component_of_wind_isobaric         (time1, isobaric3, latitude, longitude) float32 ...
    Geopotential_height_isobaric         (time1, isobaric3, latitude, longitude) float32 ...
    LatLon_361X720-0p25S-180p00E         int32 ...
Attributes: (12/13)
    Originating_or_generating_Center:                                        ...
    Originating_or_generating_Subcenter:                                     ...
    GRIB_table_version:                                                      ...
    Type_of_generating_process:                                              ...
    Analysis_or_forecast_generating_process_identifier_defined_by_originating...
    Conventions:                                                             ...
    ...                                                                                ...
    featureType:                                                             ...
    History:                                                                 ...
    geospatial_lat_min:                                                      ...
    geospatial_lat_max:                                                      ...
    geospatial_lon_min:                                                      ...
    geospatial_lon_max:                                                      ...

In [ ]:
data.isobaric3

<xarray.DataArray 'isobaric3' (isobaric3: 31)>
array([   100.,    200.,    300.,    500.,    700.,   1000.,   2000.,   3000.,
         5000.,   7000.,  10000.,  15000.,  20000.,  25000.,  30000.,  35000.,
        40000.,  45000.,  50000.,  55000.,  60000.,  65000.,  70000.,  75000.,
        80000.,  85000.,  90000.,  92500.,  95000.,  97500., 100000.])
Coordinates:
    reftime    datetime64[ns] 2017-09-05T12:00:00
  * isobaric3  (isobaric3) float64 100.0 200.0 300.0 ... 9.5e+04 9.75e+04 1e+05
Attributes:
    units:     Pa
    positive:  down

In [ ]:
#!conda install --yes -c conda-forge wrf-python
from wrf import cape_2d

In [ ]:
pres_hpa_ = data.isobaric3/100
tkel_ = data.Temperature_isobaric
qv_ = data.Relative_humidity_isobaric
height_ = data.Geopotential_height_isobaric
terrain_  = 500.0
psfc_hpa_ = 1000.0
ter_follow_  = False
meta_  = False


In [ ]:
tkel.values.shape

(9, 31, 81, 131)

In [ ]:
height_.values

array([[[[48110.49    , 48110.49    , 48110.97    , ..., 48174.33    ,
          48174.01    , 48173.53    ],
         [48103.45    , 48104.57    , 48104.41    , ..., 48173.37    ,
          48172.89    , 48172.57    ],
         [48098.01    , 48098.17    , 48098.49    , ..., 48171.77    ,
          48171.45    , 48171.13    ],
         ...,
         [47706.49    , 47706.33    , 47706.01    , ..., 47786.81    ,
          47789.69    , 47792.09    ],
         [47707.93    , 47707.61    , 47706.65    , ..., 47787.77    ,
          47790.65    , 47793.21    ],
         [47708.57    , 47707.93    , 47707.13    , ..., 47788.41    ,
          47791.45    , 47794.01    ]],

        [[42879.523   , 42880.004   , 42880.645   , ..., 42952.324   ,
          42952.004   , 42951.523   ],
         [42875.047   , 42876.324   , 42876.805   , ..., 42952.965   ,
          42952.645   , 42952.164   ],
         [42872.004   , 42872.484   , 42873.125   , ..., 42953.125   ,
          42952.805   , 42952.484

In [ ]:
MCAPE, MCIN, LCL, LFC = cape_2d(pres_hpa=pres_hpa_.values \
                                , tkel=tkel_.values[0,:,0,0], \
                                qv=qv_.values[0,:,0,0], \
                                height=height_.values[0,:,0,0], 
                                terrain=terrain_, psfc_hpa=psfc_hpa_, \
                                ter_follow=ter_follow_, missing=9.969209968386869e+36, meta=meta_)

In [ ]:
LCL

masked_array(data=[[1592.57080078125]],
             mask=[[False]],
       fill_value=9.969209968386869e+36)

In [ ]:
LFC

masked_array(data=[[47610.48828125]],
             mask=[[False]],
       fill_value=9.969209968386869e+36)

In [ ]:
MCIN

masked_array(data=[--],
             mask=[ True],
       fill_value=9.969209968386869e+36,
            dtype=float64)

In [ ]:
# A smaller difference between the LCL and LFC (LCL-LFC) is conducive to the rapid formation of thunderstorms.